# Analysis of FAIR metrics

In this notebook, the retrieved results of the FAIR metrics collection is analyzed.

In [ ]:
from datetime import date
import statistics
from collections import Counter
from pathlib import Path
import os

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from pandas.plotting import table 
import matplotlib.pyplot as plt
import matplotlib.table as table
import seaborn as sns
from scipy import stats

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams.update({'font.size': 22})

Folders and constants

In [ ]:
# subset variable is used to run the notebook for all users, only individual 
# accounts or organisations (group)
subset = "all"  # "user", "group", "all"

# todo make this into something nice
how_fair_is = pd.read_csv("results/repositories_howfairis.csv")
languages = pd.read_csv("results/languages.csv")
contributors = pd.read_csv("results/contributors.csv")
topics = pd.read_csv("results/topics.csv")
installations = pd.read_csv("results/download_stats.csv")


# file paths
fp_repos = Path("..", "collect_repositories", "results", "repositories_filtered.csv")
fp_figs = Path("figs")

# create output folder if not exists
fp_figs.mkdir(exist_ok=True)

In [ ]:
subset

## Load filtered repositories
Load (manually) filtered repositories into notebook.

In [ ]:
repos = pd.read_csv(fp_repos)

## Filtering for research groups or private users only

In [ ]:
users = pd.read_excel("../collect_users/results/users_enriched.xlsx")

In [ ]:
def add_research_group_filter(dataset):
    user_is_research_group = []
    for row in dataset.iterrows():
        try:
            owner = row[1]['owner']
        except:
            owner = row[1]['html_url_repository'].split('/')[-2]
            
            
      
        row_users = users.loc[users['user_id'] == owner]
        if len(row_users['is_research_group']) == 0:
            user_is_research_group.append(False)
        for i in row_users['is_research_group']:
            if i == 1:
                user_is_research_group.append(True)
                break
            else:
                user_is_research_group.append(False)
                break
                    
    return user_is_research_group
    

In [ ]:
repos['is_research_group'] = add_research_group_filter(repos)
contributors['is_research_group'] = add_research_group_filter(contributors)
topics['is_research_group'] = add_research_group_filter(topics)
languages['is_research_group'] = add_research_group_filter(languages)
how_fair_is['is_research_group'] = add_research_group_filter(how_fair_is)

## Subset repositories
Make a subset of repositories on "user", "group", or "all".

- "user" - Account of individual researcher
- "group" - Account of research group
- "all" - Both individual researcher or research group

In [ ]:
if subset == "user":
    repos = repos[~repos['is_research_group']]
    contributors = contributors[~contributors['is_research_group']]
    topics = topics[~topics['is_research_group']]
    languages = languages[~languages['is_research_group']]
    how_fair_is = how_fair_is[~how_fair_is['is_research_group']]
elif subset == "group":
    repos = repos[repos['is_research_group']]
    contributors = contributors[contributors['is_research_group']]
    topics = topics[topics['is_research_group']]
    languages = languages[languages['is_research_group']]
    how_fair_is = how_fair_is[how_fair_is['is_research_group']]

## Datasets

Using the github API and the Howfairis software, we collected data on the repositories, their FAIRness, the languages used in the repository, the amount of contributors for each repository and the topics mentoined in each repository. 

The Howfairis software gives  boolean value on five measures of FAIRness for each repository. These five measures consist of having a repository (which is always True for our dataset), having a license, being available in a registry, allowing citation of the software and following the FAIR software quality checklist. Below, the first five rows of the datasets are displayed. 

In [ ]:
repos

In [ ]:
how_fair_is

In [ ]:
languages

In [ ]:
contributors

In [ ]:
topics

In [ ]:
print("The repository data from Github contains the following columns:")
print(repos.columns)

## Sofware licenses 

A license is important because it shows that people have permission to use and modify your code. The plots below show the licenses used in the collection. 


In [ ]:
# short names for licenses
licenses_abbr = {
    'MIT License': 'MIT',
    'GNU General Public License v3.0': 'GPLv3',
    'Other': 'Other',
    'Apache License 2.0': 'Apache 2.0',
    'GNU General Public License v2.0': 'GPLv2',
    'BSD 3-Clause "New" or "Revised" License': 'BSD 3-Clause',
    'Creative Commons Attribution 4.0 International': 'CC-BY',
    'Creative Commons Zero v1.0 Universal': 'CC0',
    'GNU Lesser General Public License v3.0': 'LGPLv3',
    'The Unlicense': 'Unlicense',
    'GNU Affero General Public License v3.0': 'AGPPLv3',
    'BSD 2-Clause "Simplified" License': 'BSD 2-Clause',
    'Mozilla Public License 2.0': 'MPL 2.0',
    'GNU Lesser General Public License v2.1': 'LGPLv2.1',
    'Creative Commons Attribution Share Alike 4.0 International': 'CC-BY-SA',
    'ISC License': 'ISC'
    
}

In [ ]:
# # all licences
# repos["license"].value_counts().plot(kind='bar', title = 'All licences')
# plt.savefig(Path(fp_figs, f'licenses_all_{subset}.png'), bbox_inches="tight")


In [ ]:
# top most used 10 licences
top_10_licenses = repos["license"].value_counts().head(10)
print(top_10_licenses)

In [ ]:
ax = sns.barplot(top_10_licenses.index, top_10_licenses)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
ax.set_ylabel("Count")
ax.set_title("Top 10 most popular licenses")

plt.savefig(Path(fp_figs, f'licenses_first10_{subset}.png'), bbox_inches="tight")

In [ ]:
top_10_licenses_short = repos["license"].replace(licenses_abbr).value_counts().head(10)
print(top_10_licenses_short)

# first 10 licences with short names
ax = sns.barplot(top_10_licenses_short.index, top_10_licenses_short)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
ax.set_ylabel("Count")
ax.set_title("Top 10 most popular licenses")

plt.savefig(Path(fp_figs, f'licenses_short_first10_{subset}.png'), bbox_inches="tight")

In [ ]:
# first 10 licences with short names
ax = sns.barplot(top_10_licenses_short.index, top_10_licenses_short / top_10_licenses_short.sum() * 100)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
ax.set_ylabel("Percentage")
ax.set_title("Top 10 most popular licenses (in %)")

plt.savefig(Path(fp_figs, f'licenses_short_first10_percentage_{subset}.png'), bbox_inches="tight")

Below, the propotion of licenses compared to the total is shown, with their absolute count shown below that. . 

In [ ]:
percentage_license = round(float(repos["license"].isna().sum()/len(repos)* 100),2)
print("{} % of repositories have a license on their page".format(percentage_license))

## Languages

The programming language that is used in the repository gives information on the type of project the repository is. data analysis is mostly done in R and Python, though Python is used for other purposes as well. 

### Jupyter Notebooks

Please note that due to the way GitHub calculates the dominant language in a repository, Jupyter Notebooks tend to be the top language if used in a repository. The top language is determined by the number of characters, which is usually much larger in Jupyter Notebook files compared to for example Python scripts.

In [ ]:
repos["language"].value_counts().plot.bar(figsize = (100,50), fontsize = 100)
plt.title("Programming languages by popularity", fontsize = 100)
plt.savefig(Path(fp_figs, f'language_{subset}.png'), bbox_inches="tight")

In [ ]:
repos["language"].value_counts().head(20).plot.barh(figsize = (50,50), fontsize = 100)
plt.title("Top 20 Programming languages", fontsize = 100)
plt.gca().invert_yaxis()
plt.savefig(Path(fp_figs, f'language_top20_{subset}.png'), bbox_inches="tight")

In [ ]:
top_10_languages = repos["language"].value_counts().head(10)

ax = sns.barplot(top_10_languages.index, top_10_languages / top_10_languages.sum() * 100)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
ax.set_ylabel("Percentage")
ax.set_title("Top 10 most popular languages (in %)")

plt.savefig(Path(fp_figs, f'language_top10_{subset}.png'), bbox_inches="tight")

In [ ]:
repo_languages_rel = repos["language"].value_counts().head(10) / len(repos)
repo_languages_rel.plot.bar(figsize = (50,25), fontsize = 50)
plt.title("Top 10 programming languages (in %)", fontsize = 50)
plt.savefig(Path(fp_figs, f'language_top10_percentage_{subset}.png'), bbox_inches="tight")


## Relative occurence languages

In [ ]:
repo_languages_rel

## The following percentages represent the percentage of repositories that contains that specific language for the 10 most occuring languages:

In [ ]:
for value, language in zip(languages["language"].value_counts().head(10), languages["language"].value_counts().head(10).keys()):
    percentage = value / len(repos) * 100
    print("{} is present in {:.2f} % of all repositories". format(language, percentage))

In [ ]:
languages_rel = languages["language"].value_counts().head(10) / len(repos)
languages_rel.plot.bar(figsize = (50,25), fontsize = 50)
plt.title("Top 10 occuring programming languages (in %)", fontsize = 50)
plt.savefig(Path(fp_figs, f'occurence_language_top10_percentage_{subset}.png'), bbox_inches="tight")


In [ ]:
languages["user"] = languages["html_url_repository"].map(lambda url: url.split("/")[3]) # extract user from url
languages["repo"] = languages["html_url_repository"].map(lambda url: url.split("/")[4]) # extract user from url
unique_users = set(languages["user"])
userlang_chars = dict()
for user in unique_users:
    repos_user = languages.loc[languages['user'] == user]
    repos_user_total_chars = sum(repos_user["num_chars"])
    for index, row in repos_user.iterrows():
        key = row[1] # select language
        if key not in userlang_chars:
            userlang_chars[key] = row["num_chars"]/repos_user_total_chars
        else:
            userlang_chars[key] += row["num_chars"]/repos_user_total_chars

### The following shows a plot with stacked percentage of language usage of a user
#### E.g.: User X has 100 repos written in Python --> Add 1.0 to Python bar. User Y has 1 repo written in R and 1 in C# --> Add 0.5 to R and 0.5 to C#

In [ ]:
userlang_df = pd.DataFrame.from_dict(userlang_chars, orient="index", columns=['stacked_percentage'])
userlang_df.drop(["HTML", "TeX"], inplace = True)
userlang_df.sort_values(by="stacked_percentage", ascending = False, inplace = True)
userlang_df_top10 = userlang_df.head(10)
print(userlang_df_top10)

ax = sns.barplot(userlang_df_top10.index, userlang_df_top10["stacked_percentage"] / userlang_df_top10["stacked_percentage"].sum() * 100)
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
ax.set_ylabel("Percentage")
ax.set_title("Top 10 most popular languages by user (in %)")

plt.savefig(Path(fp_figs, f'most_occuring_languages_stacked_percent_top10_{subset}.png'), bbox_inches="tight")

## Correlation table top 10 languages

In [ ]:
languages["language"].value_counts().head(10).keys()
dict_languages_user = Counter()
dict_top_languages = dict()
for language in languages["language"].value_counts().head(10).keys():
    dict_top_languages[language] = 0

In [ ]:
dict_users = {}

In [ ]:

for html_url in repos['html_url']: 
    #print(html_url)
    rows_language_for_one_user = languages.loc[languages['html_url_repository'] == html_url]
    dict_languages_user = Counter()
    for language in languages["language"].value_counts().head(10).keys(): #set all values of languages that need to be present to create a dataset in a later stage to zero
        dict_languages_user[language] = 0
    for language in rows_language_for_one_user['language']:
        if language in dict_languages_user: #only count languages in the top 10
            dict_languages_user[language] += 1
    dict_users[html_url] = dict_languages_user #each user has a dict with 10 booleans representing whether the top 10 language is present in their repo or not


In [ ]:
languages_per_repo = pd.DataFrame.from_dict(dict_users, orient = 'index')

In [ ]:
languages_per_repo.corr().round(decimals = 2)

## Topics

Topics describe the context of the repository. 

In [ ]:
topics['topic'].value_counts().nlargest(50)

In [ ]:
topics['topic'].value_counts().nlargest(10).plot(kind = 'bar', title = 'Top 10 topics')
plt.savefig(Path(fp_figs, f'most_occuring_topics_top10_{subset}.png'), bbox_inches="tight")


## Number of  contributors

The number of contributors gives information on how many people put effort into the repository. Theoratically, it would make sense if a higher amount of contributors implied a more FAIR repository, because those are easier to find in a registery and work on because of their license. 


In [ ]:
contributors['html_url_repository'].value_counts().plot.hist(x = 'contributors', figsize = (10,10), fontsize = 12, title = 'Number of contributors')
plt.savefig(Path(fp_figs, f'contributors_{subset}.png'), bbox_inches="tight")


In [ ]:
contributors['html_url_repository'].value_counts().nlargest(20, keep = 'all')


In [ ]:
print('mean contributors: {:.2f}'.format(statistics.mean(contributors['html_url_repository'].value_counts())))
print('median contributors: {}'.format(statistics.median(contributors['html_url_repository'].value_counts())))

## Howfairis sum score descriptives and correlations

In this section, the median and meand of the howfairis sum score is shown, as well as correlations the how fair is sum score, the amount of contributors and the amount of months since the last commit

In [ ]:
how_fair_sum_scores = []
for index, row in how_fair_is.iterrows():
    how_fair_is_sum = row['howfairis_repository'] + row['howfairis_license'] +row['howfairis_registry'] + row['howfairis_citation'] + row['howfairis_checklist']
    how_fair_sum_scores.append(how_fair_is_sum)

In [ ]:
how_fair_is['how_fair_sum_scores'] = how_fair_sum_scores
print("mean: {:.2f}".format(statistics.mean(how_fair_sum_scores)))
print("median: {}".format(statistics.median(how_fair_sum_scores)))
pd.DataFrame((statistics.mean(how_fair_sum_scores), statistics.median(how_fair_sum_scores)),["mean", "median"])
how_fair_is.boxplot(column = 'how_fair_sum_scores')
plt.title('boxplot of the howfairis sumscore')
plt.savefig(Path(fp_figs, f'howfairissum_boxplot_{subset}.png'), bbox_inches="tight")


In [ ]:
# how_fair_is['how_fair_sum_scores'].value_counts().sort_index().plot.bar(title = 'Scores howfairis')
# plt.savefig(Path(fp_figs, f'howfairissum_histogram_{subset}.png'), bbox_inches="tight")


howfairis_result = how_fair_is['how_fair_sum_scores'].value_counts().sort_index()
print(howfairis_result)

ax = sns.barplot(howfairis_result.index, howfairis_result / howfairis_result.sum() * 100, palette='Blues_d')
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
ax.set_ylabel("Percentage")
ax.set_title("Score in howfairis tool (in %)")

In [ ]:
correlation_df = pd.DataFrame()
months_ago_most_recent_commit =[]
today = str(date.today())
split_today = today.split('-')
for date_item in repos['updated_at']:
    split_date = date_item.split('-')
    years_ago = int(split_today[0]) - int(split_date[0])
    months_ago = 12 * years_ago + (int(split_today[1]) - int(split_date[1]))
    months_ago_most_recent_commit.append(months_ago)
repos['months_ago_most_recent_commit'] = months_ago_most_recent_commit

In [ ]:
correlation_df['how_fair_sum_scores'] = how_fair_sum_scores
correlation_df['contributions']= contributors['contributions']
correlation_df['months_ago_most_recent_commit'] = repos['months_ago_most_recent_commit']

In [ ]:
correlation_df.corr(method='pearson').round(decimals=2)

In [ ]:
unique_users = set(how_fair_is["owner"])
unique_users
howfairis_max_user = list()
howfairis_average = list()
for user in unique_users:
    repos_user = how_fair_is.loc[how_fair_is['owner'] == user]
    howfairis_max_user.append(max(repos_user["how_fair_sum_scores"]))
    num_repos = len(repos_user.index)
    average = sum(repos_user["how_fair_sum_scores"])/num_repos
    howfairis_average.append([average, num_repos])

In [ ]:
plt.hist(howfairis_max_user, bins = range(1,7))
plt.xticks(range(1,6))
plt.ylabel('Count')
plt.xlabel('Howfairis score')
plt.title("Max howfairis score per user aggregated, " + str(len(unique_users)) + " users")
plt.savefig(Path(fp_figs, f'howfairis_stacked_max{subset}.png'), bbox_inches="tight")

In [ ]:
plt.scatter(*zip(*howfairis_average))
plt.ylabel('Number of repositories')
plt.xlabel('Average howfairis score')
plt.title("Average howfairis score per user, " + str(len(unique_users)) + " users")
plt.savefig(Path(fp_figs, f'howfairis_scatter_repos_avg{subset}.png'), bbox_inches="tight")

## Binary correlation plot between howfairis variables

A correlation table that shows the correlation between the howfairis measures.

In [ ]:
correlation_how_fair_is = pd.DataFrame()

#correlation_how_fair_is['howfairis_repository'] = how_fair_is['howfairis_repository']
correlation_how_fair_is['howfairis_license'] = how_fair_is['howfairis_license']
correlation_how_fair_is['howfairis_registry'] = how_fair_is['howfairis_registry']
correlation_how_fair_is['howfairis_citation'] = how_fair_is['howfairis_citation']
correlation_how_fair_is['howfairis_checklist'] = how_fair_is['howfairis_checklist']



In [ ]:
correlation_how_fair_is.corr(method='pearson').round(decimals=2) #howfairis_repository is all true, so has no variance

## Programming language and Howfairis

Different programming languages need different licences. For some languages (e.g. python), these are more complex. How does this influence the HowFairis score?

In [ ]:
index_how_fair_is = 0
language_how_fair_is_df = []
#get a list with the how-fair-is scores added to the language dataset
for index,row in languages.iterrows():
    if how_fair_is['html_url'][index_how_fair_is] != languages['html_url_repository'][index]:
        index_how_fair_is += 1 
    sum_score = how_fair_is['how_fair_sum_scores'][index_how_fair_is]
    language_how_fair_is_df.append(sum_score)
regression_data_how_fair_is_language = pd.DataFrame()
language_data_dummies = pd.get_dummies(languages['language'])  #one hot-encoding for language dataset
regression_data_how_fair_is_language['how_fair_is_sum'] = language_how_fair_is_df
languages['how_fair_sum_scores'] = regression_data_how_fair_is_language #add scores to language dataset

In [ ]:
train_size = int(len(languages) * 0.8) #training size is 2227, testing size is  557
regression_how_fair_is_language_train_X = language_data_dummies[:train_size]
regression_how_fair_is_language_train_y = regression_data_how_fair_is_language['how_fair_is_sum'][:train_size]
regression_how_fair_is_language_test_X = language_data_dummies[train_size:]
regression_how_fair_is_language_test_y = regression_data_how_fair_is_language['how_fair_is_sum'][train_size:]

In [ ]:
#train regression
regression_language_how_fair_is = LinearRegression()
regression_language_how_fair_is.fit(regression_how_fair_is_language_train_X, regression_how_fair_is_language_train_y)

# Make predictions using the testing set
language_how_fair_is_y_pred = regression_language_how_fair_is.predict(regression_how_fair_is_language_test_X)

# The coefficients
#print('Coefficients: \n', regression_language_how_fair_is.coef_)

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(regression_how_fair_is_language_test_y, language_how_fair_is_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(regression_how_fair_is_language_test_y, language_how_fair_is_y_pred))


In [ ]:
language_subset = language_data_dummies[['Python', 'Shell', 'R', 'JavaScript', 'HTML']] #top 5 languages


In [ ]:
train_size = int(len(language_subset) * 0.8)
regression_how_fair_is_language_train_X = language_subset[:train_size]
regression_how_fair_is_language_train_y = regression_data_how_fair_is_language['how_fair_is_sum'][:train_size]
regression_how_fair_is_language_test_X = language_subset[train_size:]
regression_how_fair_is_language_test_y = regression_data_how_fair_is_language['how_fair_is_sum'][train_size:]

In [ ]:
#train regression
regression_language_how_fair_is = LinearRegression()
regression_language_how_fair_is.fit(regression_how_fair_is_language_train_X, regression_how_fair_is_language_train_y)

# Make predictions using the testing set
language_how_fair_is_y_pred = regression_language_how_fair_is.predict(regression_how_fair_is_language_test_X)

# The coefficients
print('Coefficients: \n', regression_language_how_fair_is.coef_)

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(regression_how_fair_is_language_test_y, language_how_fair_is_y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(regression_how_fair_is_language_test_y, language_how_fair_is_y_pred))


A simple regression for predicting the how fair is score from  all or a subset of languages does not seem to yield any results 

## Examples of high howfairis scores 

In this section, some links to the pages that have a high howfairis score are shown. 

In [ ]:
how_fair_is_high_scores = how_fair_is[how_fair_is['how_fair_sum_scores'] > 3].sort_values('how_fair_sum_scores', ascending=False)

how_fair_is_high_scores[['html_url', 'how_fair_sum_scores']]

## Re-use of other projects and articles

Are there competitors?
How do they relate to each other?

## Installation statistics

In [ ]:
installations[installations["repository_name"] != "pcalg"].sort_values("last_month", ascending=False).reset_index(drop=True).drop("date", axis=1)